In [8]:
# Imports
from datetime import datetime, timedelta
import os
import json
import glob
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import pandas as pd
from scipy.stats import iqr
import shutil
import zipfile
from scipy.interpolate import interp1d
from scipy.ndimage import uniform_filter1d
from pathlib import Path
import geopandas

%matplotlib inline


ModuleNotFoundError: No module named 'geopandas'

In [3]:
# load data
path_data = os.path.join(os.getcwd(), "data/")
fname_data = os.path.join(path_data, f"data.csv")
df_data = pd.read_csv(fname_data)

# sort
location = df_data["Geographical location"].values
PET_participants = df_data["PET participants"].values
MRI_participants = df_data["MRI participants"].values
avg_age = df_data["Average Age (Years)"].values
male_participants = df_data["Male"].values
female_participants = df_data["Female"].values
caucasian_participants = df_data["Caucasian"].values
AfAm_participants = df_data["African-American"].values
asian_participants = df_data["Asian"].values
other_participants = df_data["Other"].values
NA_participants = df_data["Not specified"].values
AsA_participants = df_data["Asian American"].values
midE_participants = df_data["Middle Eastern"].values
carribean_participants = df_data["Carribean"].values
jewish_participants = df_data["Jewish"].values
CauHisp_participants = df_data["Caucasian-Hispanic"].values


In [7]:
import geopandas

cities = geopandas.read_file(geopandas.datasets.get_path('naturalearth_cities'))
# adding a column with random values for the size
cities['values'] = np.abs(np.random.randn(len(cities))) * 50

cities.plot(markersize=cities['values'])

ModuleNotFoundError: No module named 'geopandas'